<a href="https://colab.research.google.com/github/elephant-xyz/notebook/blob/main/County_data_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Welcome to step 3 of Elephant Mining
  by reaching this step you have successfully minted your first data group (Root group), This notebook will Use your seed data To Perpare County data group and will be validated and ready for blockchain submissions.

  ## What This Notebook Does:
  This interactive notebook automates:


-   Download properties information from County website using the Http request provided in the seed data
-   Normalize address for each property Using [OpenAddress](https://batch.openaddresses.io/) as a source of address information
- Converting retrieved properties data to lexicon format
- Validating against Elephant schemas
- Uploading to IPFS via Pinata
- Preparing transaction data for blockchain submission

## What You'll Do

1. ** upload .env file** (1 minute)
  - `.env` a file that contains your API keys a and credentials, It will be used to securely load the following environment variables


2.  ** Download properties data** (1 minute)
  - Upload upload-results.csv that was generated from the previous step using [Oracle notebook](https://colab.research.google.com/drive/14tSNSP8Pe-mY4VwX9JhXgfyOvzmN3kC0?usp=sharing#scrollTo=OFKp4E49651Z)
  - Run Downloading step
  - Input folder generated with all properties information
3.   ** Download County Addresses from Open address ** (3 minute)
  - Download address file from Open address
  - Run address normalization step
4. ** Transform Data **
  - Enter your OpenAI Key
  - Run AI Agent to convert the data
  - Auto validation against [lexicon.elephant.xyz](https://lexicon.elephant.xyz/) schemas
  - Auto Upload to IPFS and recieve content identifiers (CIDs)
  - Generate submit-results.csv
  - Download for oracle portal
5. **Submit to Blockchain** (2 minutes)

  - Visit oracle.elephant.xyz
  - Upload submit-results.csv file
  - Confirm MetaMask transactions


Your Impact
Each submission contributes to consensus. When three oracles submit matching data hashes, the data becomes blockchain truth and participants receive vMahout governance tokens.

Let's begin.







###Create account on OpenAI
1.   visit [OpenAI](https://platform.openai.com/)
2.   **SingUp** or **Login** Once logged in, you’ll land on the API Dashboard.
3. Click on start building and your details then click on **Create organization**
4. Click I'll invite my friend later
5. Now Make your first API call Step, Name your key and project then click on **Generate API key **
6. Copy your key then click continue
7. select your amount of credit to purchase, you will need 15$ for AI Agent
8. Add you payment method
9. Paste your api key in the next step

## Step 1: Upload .env file and upload-results.csv


| Variable Name           | Purpose                     |
|-------------------------|-----------------------------|
| `OPENAI_API_KEY`        | Access to OpenAI API        |
| `PINATA_JWT`     | Access to pinata key              |


- Click the **folder icon** 📂 in the left sidebar to open the file browser.
- Then click the **"Upload"** button and choose your `.env` and `upload-results.csv`


```env
# example of .env file
OPENAI_API_KEY=sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXX
PINATA_JWT=xxxxx
```

In [ ]:
# @title ## Step 2: Run to download Properties informations from County Appraiser Site

import pandas as pd
import requests
import json
import logging
import csv
import os
import time
from urllib.parse import urlencode
from typing import Optional, Dict, Any
import traceback

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class PropertyDataProcessor:
    def __init__(self, input_csv_path: str = "upload-results.csv", seed_csv_path: str = "seed.csv"):
        self.input_csv_path = input_csv_path
        self.seed_csv_path = seed_csv_path
        self.ipfs_gateways = [
            "https://ipfs.io/ipfs/",
            "https://gateway.pinata.cloud/ipfs/",
            "https://cloudflare-ipfs.com/ipfs/",
            "https://dweb.link/ipfs/",
            "https://ipfs.infura.io/ipfs/"
        ]

    def fetch_from_ipfs(self, cid: str) -> Optional[Dict[Any, Any]]:
        """Fetch data from IPFS using the provided CID with multiple gateway fallback."""
        for gateway in self.ipfs_gateways:
            try:
                url = f"{gateway}{cid}"
                logger.info(f"Trying to fetch {cid} from {gateway}")
                response = requests.get(url, timeout=10)
                response.raise_for_status()
                return response.json()
            except Exception as e:
                logger.warning(f"Error fetching from {gateway}: {e}")
                continue

        logger.error(f"Failed to fetch data from IPFS CID {cid} from all gateways")
        return None

    def trace_ipfs_chain(self, data_cid: str) -> Optional[Dict[Any, Any]]:
        """Trace through the IPFS chain to get the final property data."""

        # Step 1: Fetch the initial data using dataCid
        logger.info(f"Step 1: Fetching initial data from dataCid: {data_cid}")
        initial_data = self.fetch_from_ipfs(data_cid)
        if not initial_data:
            return None

        # Step 2: Extract property_seed CID from relationships
        try:
            property_seed_cid = initial_data["relationships"]["property_seed"]["/"]
            logger.info(f"Step 2: Found property_seed CID: {property_seed_cid}")
        except KeyError as e:
            logger.error(f"Could not find property_seed CID in initial data: {e}")
            return None

        # Step 3: Fetch property_seed data
        logger.info(f"Step 3: Fetching property_seed data from: {property_seed_cid}")
        property_seed_data = self.fetch_from_ipfs(property_seed_cid)
        if not property_seed_data:
            return None

        # Step 4: Extract "to" CID from property_seed data
        try:
            to_cid = property_seed_data["to"]["/"]
            logger.info(f"Step 4: Found 'to' CID: {to_cid}")
        except KeyError as e:
            logger.error(f"Could not find 'to' CID in property_seed data: {e}")
            return None

        # Step 5: Fetch final property data
        logger.info(f"Step 5: Fetching final property data from: {to_cid}")
        final_data = self.fetch_from_ipfs(to_cid)

        return final_data

    def create_seed_csv(self):
        """Read the input CSV, trace IPFS chain, and create seed.csv."""

        # Read the input CSV
        try:
            df = pd.read_csv(self.input_csv_path)
            logger.info(f"Loaded {len(df)} records from {self.input_csv_path}")
        except Exception as e:
            logger.error(f"Error reading CSV file: {e}")
            return False

        # Prepare output data
        output_rows = []

        for index, row in df.iterrows():
            data_cid = row['dataCid']
            logger.info(f"Processing row {index + 1}: {data_cid}")

            # Trace the IPFS chain
            final_data = self.trace_ipfs_chain(data_cid)

            if final_data:
                try:
                    # Extract data for CSV
                    parcel_id = final_data.get('request_identifier', '')
                    address = final_data.get('full_address', '')
                    county = final_data.get('county_jurisdiction', '')

                    # Extract HTTP request details
                    http_request = final_data.get('source_http_request', {})
                    method = http_request.get('method', '')
                    url = http_request.get('url', '')
                    multi_value_query_string = http_request.get('multiValueQueryString', {})

                    # Convert multiValueQueryString to JSON string for CSV
                    multi_value_query_string_str = json.dumps(multi_value_query_string) if multi_value_query_string else ''

                    # Create output row
                    output_row = {
                        'parcel_id': parcel_id,
                        'Address': address,
                        'method': method,
                        'headers': '',  # Empty as per example
                        'url': url,
                        'multiValueQueryString': multi_value_query_string_str,
                        'body': '',  # Empty as per example
                        'json': '',  # Empty as per example
                        'source_identifier': parcel_id,  # Same as parcel_id based on example
                        'County': county
                    }

                    output_rows.append(output_row)
                    logger.info(f"Successfully processed parcel ID: {parcel_id}")

                except Exception as e:
                    logger.error(f"Error processing final data for row {index + 1}: {e}")
            else:
                logger.error(f"Failed to trace IPFS chain for row {index + 1}")

        # Create output DataFrame and save to CSV
        if output_rows:
            output_df = pd.DataFrame(output_rows)
            output_df.to_csv(self.seed_csv_path, index=False)
            logger.info(f"Created {self.seed_csv_path} with {len(output_rows)} records")
            print(f"Successfully created {self.seed_csv_path} with {len(output_rows)} records")
            return True
        else:
            logger.error("No data was successfully processed")
            print("No data was successfully processed")
            return False

    def create_output_directory(self):
        """Create the input directory if it doesn't exist"""
        if not os.path.exists('input'):
            os.makedirs('input')
            logger.info("Created 'input' directory")

    def parse_multi_value_query_string(self, query_string_json):
        """Parse the multiValueQueryString JSON and convert to URL parameters"""
        try:
            if not query_string_json or query_string_json.strip() == '':
                return {}

            query_data = json.loads(query_string_json)
            # Convert multi-value query string to regular query parameters
            params = {}
            for key, values in query_data.items():
                if isinstance(values, list) and len(values) > 0:
                    params[key] = values[0]  # Take the first value
                else:
                    params[key] = values
            return params
        except json.JSONDecodeError as e:
            logger.error(f"Error parsing query string JSON: {e}")
            return {}

    def make_request(self, row):
        """Make HTTP request based on CSV row data"""
        try:
            parcel_id = row['parcel_id']
            address = row['Address']
            method = row['method'].upper()
            url = row['url']
            query_params = self.parse_multi_value_query_string(row['multiValueQueryString'])

            logger.info(f"Processing parcel {parcel_id} at {address}")

            # Use headers from CSV if provided, otherwise use minimal headers
            request_headers = {}
            if row.get('headers') and row['headers'].strip():
                try:
                    request_headers = json.loads(row['headers'])
                except json.JSONDecodeError:
                    logger.warning(f"Invalid headers JSON for parcel {parcel_id}, using minimal headers")

            # If no headers provided or parsing failed, use minimal headers
            if not request_headers:
                request_headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
                }

            # Make the request
            if method == 'GET':
                response = requests.get(url, params=query_params, headers=request_headers, timeout=30)
            elif method == 'POST':
                # Handle POST request with body if provided
                post_data = {}
                if row.get('body') and row['body'].strip():
                    try:
                        post_data = json.loads(row['body'])
                    except json.JSONDecodeError:
                        logger.warning(f"Invalid body JSON for parcel {parcel_id}, using empty body")

                # Handle JSON data if provided
                if row.get('json') and row['json'].strip():
                    try:
                        json_data = json.loads(row['json'])
                        response = requests.post(url, params=query_params, headers=request_headers, json=json_data, timeout=30)
                    except json.JSONDecodeError:
                        logger.warning(f"Invalid JSON data for parcel {parcel_id}, using form data")
                        response = requests.post(url, params=query_params, headers=request_headers, data=post_data, timeout=30)
                else:
                    response = requests.post(url, params=query_params, headers=request_headers, data=post_data, timeout=30)
            else:
                logger.warning(f"Unsupported method {method} for parcel {parcel_id}")
                return False

            response.raise_for_status()

            # Save the HTML content
            filename = f"input/{parcel_id}.html"
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(response.text)

            logger.info(f"Successfully saved {filename}")
            return True

        except requests.exceptions.RequestException as e:
            logger.error(f"Request failed for parcel {parcel_id}: {e}")
            return False
        except Exception as e:
            logger.error(f"Unexpected error processing parcel {parcel_id}: {e}")
            return False

    def download_property_data(self):
        """Read seed CSV and download property data for each parcel"""
        successful_downloads = 0
        failed_downloads = 0

        try:
            with open(self.seed_csv_path, 'r', newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)

                # Print available columns for debugging
                logger.info(f"Available columns: {reader.fieldnames}")

                # Verify required columns exist
                required_columns = ['parcel_id', 'Address', 'method', 'url', 'multiValueQueryString']
                missing_columns = [col for col in required_columns if col not in reader.fieldnames]
                if missing_columns:
                    logger.error(f"Missing required columns: {missing_columns}")
                    return False

                logger.info(f"Starting to process seed CSV file: {self.seed_csv_path}")

                # Convert reader to list to see total count
                rows = list(reader)
                total_rows = len(rows)
                logger.info(f"Found {total_rows} rows to process")

                for row_num, row in enumerate(rows, start=1):
                    logger.info(f"Processing row {row_num}/{total_rows} - Parcel: {row.get('parcel_id', 'Unknown')}")

                    try:
                        if self.make_request(row):
                            successful_downloads += 1
                            logger.info(f"✓ Successfully processed parcel {row.get('parcel_id')}")
                        else:
                            failed_downloads += 1
                            logger.error(f"✗ Failed to process parcel {row.get('parcel_id')}")
                    except Exception as e:
                        failed_downloads += 1
                        logger.error(f"✗ Exception processing parcel {row.get('parcel_id')}: {e}")

                    # Add a small delay to be respectful to the server
                    time.sleep(1)

            logger.info(f"Download complete. Successful: {successful_downloads}, Failed: {failed_downloads}")
            return True

        except FileNotFoundError:
            logger.error(f"Seed CSV file '{self.seed_csv_path}' not found")
            return False
        except Exception as e:
            logger.error(f"Error processing seed CSV file: {e}")
            logger.error(f"Full traceback: {traceback.format_exc()}")
            return False

    def run_complete_process(self):
        """Run the complete process: IPFS data fetching + property download"""
        logger.info("=" * 60)
        logger.info("STARTING COMPLETE PROPERTY DATA PROCESSING")
        logger.info("=" * 60)

        # Step 1: Create seed CSV from IPFS data
        logger.info("STEP 1: Processing IPFS data to create seed CSV...")
        if not self.create_seed_csv():
            logger.error("Failed to create seed CSV. Aborting.")
            return False

        logger.info("STEP 1 COMPLETED: Seed CSV created successfully")
        logger.info("-" * 40)

        # Step 2: Create output directory for HTML files
        logger.info("STEP 2: Creating output directory...")
        self.create_output_directory()
        logger.info("STEP 2 COMPLETED: Output directory ready")
        logger.info("-" * 40)

        # Step 3: Download property data
        logger.info("STEP 3: Downloading property data from county websites...")
        if not self.download_property_data():
            logger.error("Failed to download property data.")
            return False

        logger.info("STEP 3 COMPLETED: Property data download finished")
        logger.info("=" * 60)
        logger.info("COMPLETE PROCESS FINISHED SUCCESSFULLY")
        logger.info("=" * 60)
        return True


def main():
    """Main function to run the complete property data processor"""

    # Initialize the processor with default file paths
    # You can modify these paths as needed
    processor = PropertyDataProcessor(
        input_csv_path="upload-results.csv",  # Input CSV with dataCid column
        seed_csv_path="seed.csv"              # Output seed CSV and input for downloads
    )

    # Run the complete process
    success = processor.run_complete_process()

    if success:
        print("\n🎉 SUCCESS: Complete property data processing finished!")
        print("- Seed CSV has been created with property request details")
        print("- Property HTML files have been downloaded to 'input/' directory")
    else:
        print("\n❌ FAILED: Property data processing encountered errors")
        print("Check the logs above for detailed error information")


if __name__ == "__main__":
    main()

## Step 3: Download Open Address file for Broward County


1. Head to [OpenAddress](https://openaddresses.io/)
2. CLick on View download options
3. Search for us/fl/(the county in the seed.csv) example: us/fl/broward or us/fl/palm_beach_county
4. click on the arrow ▶ to expand and then click on download button on the left ↓
5. upload the file into Notebook same way you did for seed.csv
5. wait until it is fully uploaded before moving to step 4, you can watch for the upload status in the botton left corner of the notebook



In [ ]:
# @title Step 4: Enter your OpenAddress File name (WAIT until it is fully uploaded)
import os
import sys
File_Name = "palm_beach.geojson" # @param {"type":"string"}
os.environ["OpenAddress"] = File_Name
def wait_for_upload(filename):
    if not os.path.exists(filename):
        print(f"❌ File {filename} not found!")
        return False

    print("Checking if upload is complete...")
    size1 = os.path.getsize(filename)
    time.sleep(5)  # Wait 5 seconds
    size2 = os.path.getsize(filename)

    if size1 == size2 and size1 > 0:
        print(f"✅ Upload complete! File size: {size1:,} bytes")
        return True
    else:
        print("❌ File still uploading, wait longer")
        return False

# Check the upload
if wait_for_upload(File_Name):
    print("Ready to proceed!")
else:
    print("PLease wait for blue circle in the bottom left of the screen. If you don't see the circle, that means that you didn't upload the file.")


In [ ]:
# @title Step 5: Run to normalize addresse using Open Address file
import os
import csv
import json
import re
from pathlib import Path
from collections import defaultdict
from difflib import SequenceMatcher

# Define directional and suffix standardization
DIRECTIONAL_PREFIXES = {
    "N": "NORTH", "S": "SOUTH", "E": "EAST", "W": "WEST",
    "NE": "NORTHEAST", "NW": "NORTHWEST", "SE": "SOUTHEAST", "SW": "SOUTHWEST"
}

USPS_SUFFIXES = {
    "ALLEE": "ALY", "ALLEY": "ALY", "ALLY": "ALY", "ALY": "ALY", "ANEX": "ANX", "ANNEX": "ANX", "ANNX": "ANX", "ANX": "ANX",
    "ARC": "ARC", "ARCADE": "ARC", "AV": "AVE", "AVE": "AVE", "AVEN": "AVE", "AVENU": "AVE", "AVENUE": "AVE", "AVN": "AVE", "AVNUE": "AVE",
    "BAYOO": "BYU", "BAYOU": "BYU", "BCH": "BCH", "BEACH": "BCH", "BEND": "BND", "BND": "BND", "BLF": "BLF", "BLUF": "BLF", "BLUFF": "BLF", "BLUFFS": "BLFS",
    "BOT": "BTM", "BTM": "BTM", "BOTTM": "BTM", "BOTTOM": "BTM", "BLVD": "BLVD", "BOUL": "BLVD", "BOULEVARD": "BLVD", "BOULV": "BLVD",
    "BR": "BR", "BRNCH": "BR", "BRANCH": "BR", "BRDGE": "BRG", "BRG": "BRG", "BRIDGE": "BRG", "BRK": "BRK", "BROOK": "BRK", "BROOKS": "BRKS",
    "BURG": "BG", "BURGS": "BGS", "BYP": "BYP", "BYPA": "BYP", "BYPAS": "BYP", "BYPASS": "BYP", "BYPS": "BYP", "CAMP": "CP", "CP": "CP", "CMP": "CP",
    "CANYN": "CYN", "CANYON": "CYN", "CNYN": "CYN", "CAPE": "CPE", "CPE": "CPE", "CAUSEWAY": "CSWY", "CAUSWA": "CSWY", "CSWY": "CSWY",
    "CEN": "CTR", "CENT": "CTR", "CENTER": "CTR", "CENTR": "CTR", "CENTRE": "CTR", "CNTER": "CTR", "CNTR": "CTR", "CTR": "CTR", "CENTERS": "CTRS",
    "CIR": "CIR", "CIRC": "CIR", "CIRCL": "CIR", "CIRCLE": "CIR", "CRCL": "CIR", "CRCLE": "CIR", "CIRCLES": "CIRS", "CLF": "CLF", "CLIFF": "CLF",
    "CLFS": "CLFS", "CLIFFS": "CLFS", "CLB": "CLB", "CLUB": "CLB", "COMMON": "CMN", "COMMONS": "CMNS", "COR": "COR", "CORNER": "COR", "CORNERS": "CORS", "CORS": "CORS",
    "COURSE": "CRSE", "CRSE": "CRSE", "COURT": "CT", "CT": "CT", "COURTS": "CTS", "CTS": "CTS", "COVE": "CV", "CV": "CV", "COVES": "CVS",
    "CREEK": "CRK", "CRK": "CRK", "CRESCENT": "CRES", "CRES": "CRES", "CRSENT": "CRES", "CRSNT": "CRES", "CREST": "CRST", "CROSSING": "XING", "CRSSNG": "XING", "XING": "XING",
    "CROSSROAD": "XRD", "CROSSROADS": "XRDS", "CURVE": "CURV", "DALE": "DL", "DL": "DL", "DAM": "DM", "DM": "DM", "DIV": "DV", "DIVIDE": "DV", "DV": "DV", "DVD": "DV",
    "DR": "DR", "DRIV": "DR", "DRIVE": "DR", "DRV": "DR", "DRIVES": "DRS", "EST": "EST", "ESTATE": "EST", "ESTATES": "ESTS", "ESTS": "ESTS",
    "EXP": "EXPY", "EXPR": "EXPY", "EXPRESS": "EXPY", "EXPRESSWAY": "EXPY", "EXPW": "EXPY", "EXPY": "EXPY", "EXT": "EXT", "EXTENSION": "EXT", "EXTN": "EXT", "EXTNSN": "EXT", "EXTENSIONS": "EXTS", "EXTS": "EXTS",
    "FALL": "FALL", "FALLS": "FLS", "FLS": "FLS", "FERRY": "FRY", "FRRY": "FRY", "FRY": "FRY", "FIELD": "FLD", "FLD": "FLD", "FIELDS": "FLDS", "FLDS": "FLDS",
    "FLAT": "FLT", "FLT": "FLT", "FLATS": "FLTS", "FLTS": "FLTS", "FORD": "FRD", "FRD": "FRD", "FORDS": "FRDS", "FOREST": "FRST", "FORESTS": "FRST", "FRST": "FRST",
    "FORG": "FRG", "FORGE": "FRG", "FRG": "FRG", "FORGES": "FRGS", "FORK": "FRK", "FRK": "FRK", "FORKS": "FRKS", "FRKS": "FRKS", "FORT": "FT", "FRT": "FT", "FT": "FT",
    "FREEWAY": "FWY", "FREEWY": "FWY", "FRWAY": "FWY", "FRWY": "FWY", "FWY": "FWY", "GARDEN": "GDN", "GARDN": "GDN", "GRDEN": "GDN", "GRDN": "GDN", "GARDENS": "GDNS", "GDNS": "GDNS", "GRDNS": "GDNS",
    "GATEWAY": "GTWY", "GATEWY": "GTWY", "GATWAY": "GTWY", "GTWAY": "GTWY", "GTWY": "GTWY", "GLEN": "GLN", "GLN": "GLN", "GLENS": "GLNS", "GREEN": "GRN", "GRN": "GRN", "GREENS": "GRNS",
    "GROV": "GRV", "GROVE": "GRV", "GRV": "GRV", "GROVES": "GRVS", "HARB": "HBR", "HARBOR": "HBR", "HARBR": "HBR", "HBR": "HBR", "HRBOR": "HBR", "HARBORS": "HBRS",
    "HAVEN": "HVN", "HVN": "HVN", "HT": "HTS", "HTS": "HTS", "HIGHWAY": "HWY", "HIGHWY": "HWY", "HIWAY": "HWY", "HIWY": "HWY", "HWAY": "HWY", "HWY": "HWY",
    "HILL": "HL", "HL": "HL", "HILLS": "HLS", "HLS": "HLS", "HLLW": "HOLW", "HOLLOW": "HOLW", "HOLLOWS": "HOLW", "HOLW": "HOLW", "HOLWS": "HOLW",
    "INLET": "INLT", "INLT": "INLT", "IS": "IS", "ISLAND": "IS", "ISLND": "IS", "ISLANDS": "ISS", "ISLNDS": "ISS", "ISS": "ISS", "ISLE": "ISLE", "ISLES": "ISLE",
    "JCT": "JCT", "JCTION": "JCT", "JCTN": "JCT", "JUNCTION": "JCT", "JUNCTN": "JCT", "JUNCTON": "JCT", "JCTNS": "JCTS", "JCTS": "JCTS", "JUNCTIONS": "JCTS",
    "KEY": "KY", "KY": "KY", "KEYS": "KYS", "KYS": "KYS", "KNL": "KNL", "KNOL": "KNL", "KNOLL": "KNL", "KNLS": "KNLS", "KNOLLS": "KNLS",
    "LK": "LK", "LAKE": "LK", "LKS": "LKS", "LAKES": "LKS", "LAND": "LAND", "LANDING": "LNDG", "LNDG": "LNDG", "LNDNG": "LNDG", "LANE": "LN", "LN": "LN",
    "LGT": "LGT", "LIGHT": "LGT", "LIGHTS": "LGTS", "LF": "LF", "LOAF": "LF", "LCK": "LCK", "LOCK": "LCK", "LCKS": "LCKS", "LOCKS": "LCKS",
    "LDG": "LDG", "LDGE": "LDG", "LODG": "LDG", "LODGE": "LDG", "LOOP": "LOOP", "LOOPS": "LOOP", "MALL": "MALL", "MNR": "MNR", "MANOR": "MNR", "MANORS": "MNRS", "MNRS": "MNRS",
    "MEADOW": "MDW", "MDW": "MDW", "MDWS": "MDWS", "MEADOWS": "MDWS", "MEDOWS": "MDWS", "MEWS": "MEWS", "MILL": "ML", "MILLS": "MLS", "MISSION": "MSN", "MISSN": "MSN", "MSSN": "MSN",
    "MOTORWAY": "MTWY", "MNT": "MT", "MT": "MT", "MOUNT": "MT", "MNTAIN": "MTN", "MNTN": "MTN", "MOUNTAIN": "MTN", "MOUNTIN": "MTN", "MTIN": "MTN", "MTN": "MTN", "MNTNS": "MTNS", "MOUNTAINS": "MTNS",
    "NCK": "NCK", "NECK": "NCK", "ORCH": "ORCH", "ORCHARD": "ORCH", "ORCHRD": "ORCH", "OVAL": "OVAL", "OVL": "OVAL", "OVERPASS": "OPAS", "PARK": "PARK", "PRK": "PARK", "PARKS": "PARK",
    "PARKWAY": "PKWY", "PARKWY": "PKWY", "PKWAY": "PKWY", "PKWY": "PKWY", "PKY": "PKWY", "PARKWAYS": "PKWY", "PKWYS": "PKWY", "PASS": "PASS", "PASSAGE": "PSGE", "PATH": "PATH", "PATHS": "PATH",
    "PIKE": "PIKE", "PIKES": "PIKE", "PINE": "PNE", "PINES": "PNES", "PNES": "PNES", "PLACE": "PL", "PL": "PL", "PLAIN": "PLN", "PLN": "PLN", "PLAINS": "PLNS", "PLNS": "PLNS",
    "PLAZA": "PLZ", "PLZ": "PLZ", "PLZA": "PLZ", "POINT": "PT", "PT": "PT", "POINTS": "PTS", "PTS": "PTS", "PORT": "PRT", "PRT": "PRT", "PORTS": "PRTS", "PRTS": "PRTS",
    "PR": "PR", "PRAIRIE": "PR", "PRR": "PR", "RAD": "RADL", "RADIAL": "RADL", "RADIEL": "RADL", "RADL": "RADL", "RAMP": "RAMP", "RANCH": "RNCH", "RANCHES": "RNCH", "RNCH": "RNCH", "RNCHS": "RNCH",
    "RAPID": "RPD", "RPD": "RPD", "RAPIDS": "RPDS", "RPDS": "RPDS", "REST": "RST", "RST": "RST", "RDG": "RDG", "RDGE": "RDG", "RIDGE": "RDG", "RDGS": "RDGS", "RIDGES": "RDGS",
    "RIV": "RIV", "RIVER": "RIV", "RVR": "RIV", "RIVR": "RIV", "RD": "RD", "ROAD": "RD", "ROADS": "RDS", "RDS": "RDS", "ROUTE": "RTE", "ROW": "RTE", "RUE": "RUE", "RUN": "RUN",
    "SHL": "SHL", "SHOAL": "SHL", "SHLS": "SHLS", "SHOALS": "SHLS", "SHOAR": "SHR", "SHORE": "SHR", "SHR": "SHR", "SHOARS": "SHRS", "SHORES": "SHRS", "SHRS": "SHRS",
    "SKYWAY": "SKWY", "SPG": "SPG", "SPNG": "SPG", "SPRING": "SPG", "SPRNG": "SPG", "SPGS": "SPGS", "SPNGS": "SPGS", "SPRINGS": "SPGS", "SPRNGS": "SPGS",
    "SPUR": "SPUR", "SPURS": "SPUR", "SQ": "SQ", "SQR": "SQ", "SQRE": "SQ", "SQU": "SQ", "SQUARE": "SQ", "SQRS": "SQS", "SQUARES": "SQS",
    "STA": "STA", "STATION": "STA", "STATN": "STA", "STN": "STA", "STRA": "STRA", "STRAV": "STRA", "STRAVEN": "STRA", "STRAVENUE": "STRA", "STRAVN": "STRA", "STRVN": "STRA", "STRVNUE": "STRA",
    "STREAM": "STRM", "STREME": "STRM", "STRM": "STRM", "STREET": "ST", "STRT": "ST", "ST": "ST", "STR": "ST", "STREETS": "STS",
    "SMT": "SMT", "SUMIT": "SMT", "SUMITT": "SMT", "SUMMIT": "SMT", "TER": "TER", "TERR": "TER", "TERRACE": "TER", "THROUGHWAY": "TRWY", "TRACE": "TRCE", "TRACES": "TRCE", "TRCE": "TRCE",
    "TRACK": "TRAK", "TRACKS": "TRAK", "TRAK": "TRAK", "TRK": "TRAK", "TRKS": "TRAK", "TRAFFICWAY": "TRFY", "TRAIL": "TRL", "TRAILS": "TRL", "TRL": "TRL", "TRLS": "TRL",
    "TRAILER": "TRLR", "TRLR": "TRLR", "TRLRS": "TRLR", "TUNEL": "TUNL", "TUNL": "TUNL", "TUNLS": "TUNL", "TUNNEL": "TUNL", "TUNNELS": "TUNL", "TUNNL": "TUNL",
    "TRNPK": "TPKE", "TURNPIKE": "TPKE", "TURNPK": "TPKE", "UNDERPASS": "UPAS", "UN": "UN", "UNION": "UN", "UNIONS": "UNS", "VALLEY": "VLY", "VALLY": "VLY", "VLLY": "VLY", "VLY": "VLY",
    "VALLEYS": "VLYS", "VLYS": "VLYS", "VDCT": "VIA", "VIA": "VIA", "VIADCT": "VIA", "VIADUCT": "VIA", "VIEW": "VW", "VW": "VW", "VIEWS": "VWS", "VWS": "VWS",
    "VILL": "VLG", "VILLAG": "VLG", "VILLAGE": "VLG", "VILLG": "VLG", "VILLIAGE": "VLG", "VLG": "VLG", "VILLAGES": "VLGS", "VLGS": "VLGS", "VILLE": "VL", "VL": "VL",
    "VIS": "VIS", "VIST": "VIS", "VISTA": "VIS", "VST": "VIS", "VSTA": "VIS", "WALK": "WALK", "WALKS": "WALK", "WALL": "WALL", "WY": "WAY", "WAY": "WAY", "WAYS": "WAYS",
    "WELL": "WL", "WELLS": "WLS", "WLS": "WLS"
}

def parse_csv_address(address_string):
    """Parse CSV address: '1605 S US HIGHWAY 1 3E,PALM BEACH GARDENS'"""
    # Split by comma - everything after comma is city
    if ',' in address_string:
        street_part, city_part = address_string.split(',', 1)
        city = city_part.strip().lower()  # Normalize to lowercase
    else:
        street_part = address_string
        city = None

    # Parse street part
    parts = street_part.strip().upper().split()

    # Extract house number (first part should be number)
    number = None
    street_parts = []
    unit = None

    if parts and re.match(r'^\d+[A-Z]?$', parts[0]):
        number = parts[0]
        remaining_parts = parts[1:]
    else:
        remaining_parts = parts

    # Look for unit at the end (like "3E")
    if remaining_parts and re.match(r'^\d+[A-Z]$', remaining_parts[-1]):
        unit = remaining_parts[-1]
        street_parts = remaining_parts[:-1]
    else:
        street_parts = remaining_parts

    return {
        'number': number,
        'street_parts': street_parts,
        'unit': unit,
        'city': city
    }

def normalize_street_name(street_parts):
    """Normalize street name parts for comparison"""
    normalized = []
    for part in street_parts:
        if part in DIRECTIONAL_PREFIXES:
            normalized.append(DIRECTIONAL_PREFIXES[part])
        elif part in USPS_SUFFIXES:
            normalized.append(USPS_SUFFIXES[part])
        else:
            normalized.append(part)
    return ' '.join(normalized).lower()

def similarity_score(str1, str2):
    """Calculate similarity score between two strings"""
    return SequenceMatcher(None, str1.lower(), str2.lower()).ratio()

def fuzzy_street_match(csv_street, geo_street, csv_unit=None, geo_unit=None, threshold=0.7):
    """Fuzzy match street names with exact unit matching"""
    # Normalize both streets
    csv_normalized = normalize_street_name(csv_street)
    geo_normalized = normalize_street_name(geo_street.upper().split())

    # Calculate street similarity
    street_score = similarity_score(csv_normalized, geo_normalized)

    # Exact unit matching only - no fuzzy matching
    unit_match = False
    if csv_unit and geo_unit:
        csv_unit_clean = str(csv_unit).upper().strip()
        geo_unit_clean = str(geo_unit).upper().strip()
        unit_match = (csv_unit_clean == geo_unit_clean)
    elif not csv_unit and not geo_unit:
        # Both have no units - that's also a match
        unit_match = True
    elif not csv_unit or not geo_unit:
        # One has unit, other doesn't - still allow street match
        unit_match = None  # Neutral - don't penalize

    return street_score >= threshold, street_score, unit_match

def load_csv_records():
    """Load and parse CSV records"""
    print("Loading CSV records...")
    csv_records = []

    with open('seed.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            original_address = row['Address']
            parsed = parse_csv_address(original_address)

            record = {
                'parcel_id': row['parcel_id'],
                'original_address': original_address,
                'parsed': parsed,
                'county': row.get('County', ''),
                'street_normalized': normalize_street_name(parsed['street_parts']) if parsed['street_parts'] else ''
            }

            csv_records.append(record)

    print(f"Loaded {len(csv_records)} CSV records")
    return csv_records

def match_addresses_from_geojson():
    # Load CSV records
    csv_records = load_csv_records()

    # Read GeoJSON path from env
    geojson_path = os.environ.get("OpenAddress")
    if not geojson_path:
        print("Warning: OpenAddress environment variable not set.")
        return

    print(f"Loading GeoJSON from: {geojson_path}")

    # Process GeoJSON and match
    matches_by_parcel = defaultdict(list)
    processed_count = 0

    try:
        with open(geojson_path, 'r', encoding='utf-8') as geojsonfile:
            for line_num, line in enumerate(geojsonfile, 1):

                try:
                    feature = json.loads(line)
                    props = feature['properties']
                    coords = feature['geometry']['coordinates']

                    if not props.get('number') or not props.get('street'):
                        continue

                    geo_number = str(props['number']).upper()
                    geo_street = props['street']
                    geo_city = props.get('city', '').lower() if props.get('city') else None  # Normalize to lowercase
                    geo_unit = props.get('unit', '') if props.get('unit') else None

                    # Clean up ordinal suffixes in GeoJSON street
                    geo_street_clean = re.sub(r'(\d+)(th|st|nd|rd)', r'\1', geo_street)
                    geo_street_parts = geo_street_clean.upper().split()

                    # Match against CSV records
                    for csv_record in csv_records:
                        csv_parsed = csv_record['parsed']

                        # Check house number match (required)
                        if csv_parsed['number'] and geo_number != csv_parsed['number']:
                            continue

                        # Check city match if both have cities - be more flexible
                        city_score = None
                        if csv_parsed['city'] and geo_city:
                            # Allow fuzzy city matching with lower threshold
                            city_match, city_score, _ = fuzzy_street_match([csv_parsed['city']], geo_city, threshold=0.6)
                            # If strict city matching fails, try partial matching
                            if not city_match:
                                # Check if cities contain each other or share significant words
                                csv_city_words = set(csv_parsed['city'].split())
                                geo_city_words = set(geo_city.split())
                                common_words = csv_city_words.intersection(geo_city_words)

                                # If they share important words like palm, beach, gardens, jupiter
                                if common_words or csv_parsed['city'] in geo_city or geo_city in csv_parsed['city']:
                                    city_match = True
                                    city_score = 0.7  # Moderate score for partial match
                                else:
                                    city_match = True  # Allow mismatch for now
                                    city_score = 0.5

                        # Street matching with exact unit matching
                        street_match, street_score, unit_exact_match = fuzzy_street_match(
                            csv_parsed['street_parts'],
                            geo_street_clean,
                            csv_parsed['unit'],
                            geo_unit
                        )

                        # Filter: Only keep if street matches AND unit matches exactly (if both have units)
                        if csv_parsed['unit'] and geo_unit:
                            if not unit_exact_match:  # Skip if units don't match exactly
                                continue

                        if street_match:
                            enriched = {
                                **props,
                                "coordinates": coords,
                                "original_csv_address": csv_record['original_address'],
                                "parcel_id": csv_record["parcel_id"],
                                "county": csv_record["county"],
                                "match_scores": {
                                    "street_score": round(street_score, 3),
                                    "unit_exact_match": unit_exact_match,
                                    "city_score": round(city_score, 3) if city_score else None
                                }
                            }
                            matches_by_parcel[csv_record['parcel_id']].append(enriched)

                    processed_count += 1

                except json.JSONDecodeError:
                    continue

    except FileNotFoundError:
        print(f"Error: GeoJSON file not found at {geojson_path}")
        return

    # Save results
    possible_addresses_dir = Path("possible_addresses")
    possible_addresses_dir.mkdir(exist_ok=True)

    for parcel_id, matches in matches_by_parcel.items():
        # Sort matches by street score
        matches.sort(key=lambda x: x['match_scores']['street_score'], reverse=True)

        # Filter logic: If we have unit matches, exclude empty unit matches
        has_unit_matches = any(match.get('unit') for match in matches)

        if has_unit_matches:
            # Keep only matches with units
            matches = [match for match in matches if match.get('unit')]

        address_data = []
        for match in matches:
            # Format the data to match the desired output structure
            address_data.append({
                "number": str(match["number"]),
                "street": match["street"].title(),  # Title case for street names
                "unit": match.get("unit", ""),
                "city": match.get("city", "").title() if match.get("city") else "",
                "district": match.get("district", ""),
                "postcode": match.get("postcode", ""),
                "coordinates": match["coordinates"]
            })

        file_path = possible_addresses_dir / f"{parcel_id}.json"
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(address_data, f, indent=2)

    print(f"✅ {len(matches_by_parcel)} addresses were successfully matched")

    # Show unmatched records
    matched_parcel_ids = set(matches_by_parcel.keys())
    unmatched = [record for record in csv_records if record["parcel_id"] not in matched_parcel_ids]

    if unmatched:
        print(f"⚠️  {len(unmatched)} addresses had no matches:")
        for record in unmatched:
            print(f"  Parcel: {record['parcel_id']} - '{record['original_address']}'")

if __name__ == "__main__":
    match_addresses_from_geojson()

Loading CSV records...
Loaded 2 CSV records
Loading GeoJSON from: palm_beach.geojson
✅ 2 addresses were successfully matched


In [ ]:
# @title Step 7: Run to use AI Agent to convert properties info into validated Lexicon
!uvx --from git+https://github.com/elephant-xyz/AI-Agent test-evaluator-agent >> logs/elephant-cli.log
!npx -y @elephant-xyz/cli@latest validate-and-upload submit --output-csv submit-results.csv

In [ ]:
!npx -y @elephant-xyz/cli@latest validate-and-upload submit --output-csv submit-results.csv

## Step 8: Submitting Your Data to the Blockchain

### Submitting Your Data

After running the upload command in the notebook:

1. **Download your results file**
   - The notebook will generate `submit-results.csv`
   - This file contains your data hashes and IPFS CIDs
   - Download it to your computer

2. **Visit the Oracle Submission Portal**
   - Go to https://oracle.elephant.xyz/
   - Connect your MetaMask wallet when prompted
   - Upload your `submit-results.csv` file

3. **Submit transactions**
   - The portal will read your CSV and prepare transactions
   - Click "Submit to Contract" to begin
   - MetaMask will pop up for each data entry
   - Confirm each transaction (small gas fee applies)
   - Wait for confirmations between submissions

Once complete, your data is permanently recorded on the blockchain. You'll receive vMahout tokens as rewards after consensus is reached (when 3 different oracles submit matching data hashes).

In [ ]:
! zip -r submit.zip submit/
!rm -r owners/ data/ scripts/ logs/ results.csv submit/ submit-results.csv elephant-cli.log fact-sheet-build.log palm_beach.geojson seed.csv submit.zip submit_errors.csv submit_warnings.csv upload-results.csv
!rm -rf /root/.local/bin/fact-sheet
!rm -rf fact-sheet-template/
!rm -rf /root/.elephant-fact-sheet

rm: cannot remove 'owners/': No such file or directory
rm: cannot remove 'data/': No such file or directory
rm: cannot remove 'scripts/': No such file or directory
rm: cannot remove 'logs/': No such file or directory
rm: cannot remove 'results.csv': No such file or directory
rm: cannot remove 'submit/': No such file or directory
